<a href="https://colab.research.google.com/github/FaIIen/101Alphas/blob/main/rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
!pip install talib-binary
!pip install tushare
!git clone https://github.com/FaIIen/101Alphas.git

# 环境初始化

In [ ]:
%matplotlib inline
%load_ext tensorboard

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import gym
import yfinance as yf
import itertools
import os
import sys
import math
import torch as th
import tushare as ts
sys.path.append("../FinRL-Library")
ts.set_token('287ca801de30d29b564d0981a825f01062fb7e7888f163bc62170438')

from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize
from gym import spaces
from gym.utils import seeding
from tqdm import tqdm
from stable_baselines3 import PPO, SAC

sys.path.insert(0,'/content/101Alphas')
import utils as u
import feature_generation as fg

if not os.path.exists("./" + "tb_log"):
  os.makedirs("./" + "tb_log")

seed = 29
set_random_seed(seed)

# Tensorboard

In [ ]:
%tensorboard --logdir tb_log

# 数据下载

In [ ]:
cnstock = ["000002.sz"]

def get_date_by_delta(dt, delta):
  download_start_dt = datetime.datetime.strptime(dt, '%Y-%m-%d')
  delta = datetime.timedelta(days=delta)
  download_start_dt = download_start_dt + delta
  download_start_dt = download_start_dt.strftime('%Y-%m-%d')
  return download_start_dt

train_start_dt = "2007-01-01" # 训练开始时间
train_end_dt = "2020-12-31"
valid_start_dt = "2021-01-01" # 评测开始时间
valid_end_dt = "2022-12-31" # 评测结束时间
download_start_dt = get_date_by_delta(train_start_dt, -90)
# train_end_dt = get_date_by_delta(valid_start_dt, -50)

class MyDownloader:
  def __init__(self, start_date: str, end_date: str, ticker_list: list):
    self.start_date = start_date
    self.end_date = end_date
    self.ticker_list = ticker_list

  def fetch_data(self, proxy=None, auto_adjust=True) -> pd.DataFrame:
    data_df = pd.DataFrame()
    for tic in self.ticker_list:
      if auto_adjust:
        temp_df = ts.pro_bar(ts_code=tic, adj='hfq', start_date=self.start_date, end_date=self.end_date)
      else:
        temp_df = ts.pro_bar(ts_code=tic, adj='None', start_date=self.start_date, end_date=self.end_date)
      del temp_df['ts_code'],temp_df['pre_close'],temp_df['change'],temp_df['pct_chg'],temp_df['amount']
      # temp_df = yf.download(tic, start=self.start_date, end=self.end_date, proxy=proxy
      #                       , prepost=False, auto_adjust=auto_adjust)
      temp_df["tic"] = tic
      data_df = data_df.append(temp_df)
    data_df = data_df.reset_index(drop=True)
    data_df.columns = [
      "date",
      "open",
      "high",
      "low",
      "close",
      "volume",
      "tic"
    ]
    data_df["date"] = pd.to_datetime(data_df['date'], format="%Y%m%d")
    data_df["day"] = data_df["date"].dt.dayofweek
    data_df["date"] = data_df.date.apply(lambda x: x.strftime("%Y-%m-%d"))
    data_df.drop_duplicates(subset=['open','high','low','close','volume','tic'], keep='first', inplace=True)
    data_df.drop(data_df[data_df['volume']==0.0].index, inplace=True)
    data_df.dropna(inplace=True)
    # data_df.drop(data_df[data_df['low'] == data_df['high']].index, inplace=True)
    data_df = data_df.sort_values(by=["date", "tic"]).reset_index(drop=True)
    print(data_df.shape)
    return data_df


df = MyDownloader(start_date = download_start_dt, end_date = valid_end_dt, ticker_list = cnstock).fetch_data(auto_adjust=True)

df.sort_values(['date','tic'],ignore_index=True).head()

# 特征抽取

In [ ]:
import talib



class MyFeatureEngineer(FeatureEngineer):
  def add_user_defined_feature(self, data):
    df = data.copy()
    df['vwap'] = u.vwap(df)
    df['returns'] = u.returns(df)   
    unavailable = [48,53,56,57,58,59,63,67,69,70,76,79,80,82,83,87,89,90,91,93,97,100]
    for n in range(101):
      if n+1 in unavailable:
        continue
      name = 'alpha{}'.format(n+1)
      func = 'fg.{}'.format(name)
      temp_func = eval(func)
      df[name] = temp_func(df)

      df[name][np.isinf(df[name])] = 0.0
    del df['vwap'], df['returns']
    return df

test_fe = MyFeatureEngineer(
    use_technical_indicator=False,
    use_vix=False,
    use_turbulence=False,
    user_defined_feature=True
)
processed_full = test_fe.preprocess_data(df)
processed_full = processed_full[processed_full['date']>=train_start_dt].reset_index(drop=True)
processed_full

In [ ]:
# 标准化
indicator_list = processed_full.columns[8::].values.tolist()
print(len(indicator_list))
print(indicator_list)

for indicator in indicator_list:
  processed_full[indicator] = (processed_full[indicator] - processed_full[(processed_full['date']>train_start_dt) & (processed_full['date']<train_end_dt)][indicator].mean())/processed_full[(processed_full['date']>train_start_dt) & (processed_full['date']<train_end_dt)][indicator].std(ddof=0)

# processed_full.loc[:,indicator_list] = processed_full.loc[:,indicator_list].apply(zscore)
processed_full

In [ ]:
noadjust_df = MyDownloader(start_date = download_start_dt, end_date = valid_end_dt, ticker_list = cnstock).fetch_data(auto_adjust=False)
noadjust_df.sort_values(['date','tic'],ignore_index=True).head()

processed_full = pd.merge(noadjust_df, processed_full, on=['date', 'tic'], suffixes=['', '_tmp'])
del processed_full['open_tmp'],processed_full['high_tmp'],processed_full['low_tmp']
del processed_full['close_tmp'],processed_full['volume_tmp'],processed_full['day_tmp']
processed_full

In [ ]:
processed_full[processed_full.isnull().values].tail().T

In [ ]:
processed_full.describe().T

# RL环境env

state = [资金分布， 特征]

In [ ]:
class MyTradeEnv(StockTradingEnv):

  def __init__(
          self,
          df,
          stock_dim,
          initial_amount,
          buy_cost_pct,
          sell_cost_pct,
          state_space,
          n_action_space,
          reward_scaling,
          tech_indicator_list,
          ncpu,
          hmax,
          make_plots=False,
          print_verbosity=10,
          day=0,
          initial=True,
          model_name="",
          mode="",
          iteration="",
          n_invalid_actions=0):
    self.day = day
    self.df = df
    self.stock_dim = stock_dim
    self.initial_amount = initial_amount
    self.buy_cost_pct = buy_cost_pct
    self.sell_cost_pct = sell_cost_pct
    self.state_space = state_space
    self.n_action_space = n_action_space
    self.tech_indicator_list = tech_indicator_list
    self.reward_scaling = reward_scaling
    self.ncpu = ncpu
    self.hmax = hmax

    self.n_invalid_actions = n_invalid_actions
    self.possible_actions = np.arange(self.n_action_space)
    self.invalid_actions: List[int] = []

    self.action_space = spaces.Discrete(self.n_action_space)
    # self.action_space = spaces.Box(low=-1, high=1, shape=(self.n_action_space,))
    self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.state_space,))
    self.data = self.df.loc[self.day, :]
    self.terminal = False
    self.make_plots = make_plots
    self.print_verbosity = print_verbosity
    self.initial = initial
    self.model_name = model_name
    self.mode = mode
    self.iteration = iteration
    self.state = self._initiate_state()
    self.reward = 0
    self.cost = 0
    self.trades = 0
    self.episode = 0
    self.asset_memory = [self.initial_amount]
    self.rewards_memory = []
    self.actions_memory = []
    self.date_memory = [self._get_date()]
    self.max_total_assets = 0
    self._seed()

  def reset(self):
    self.state = self._initiate_state()
    self.asset_memory = [self.initial_amount]
    self.day = 0
    self.data = self.df.loc[self.day, :]
    self.cost = 0
    self.trades = 0
    self.terminal = False
    self.rewards_memory = []
    self.actions_memory = []
    self.date_memory = [self._get_date()]
    self.episode += 1
    self.max_total_assets = 0
    self.possible_actions = np.arange(self.n_action_space)
    self.invalid_actions: List[int] = []
    return self.state

  def action_masks(self):
    action_masks = [action not in self.invalid_actions for action in self.possible_actions]
    if self.mode == 'test':
      print("action_masks:", action_masks)
    return action_masks

  def update_action_masks(self):
    account = round(self.state[0]/0.2) + 5
    invalid_actions = []
    for i in range(11):
      if i+account >= 10 and i+account<=15:
        continue
      invalid_actions.append(i)
    self.invalid_actions = np.array(invalid_actions)
    if self.mode == 'test':
      print("invalid_actions:", self.invalid_actions, " self.state[0]: ", self.state[0])

  def _sell_stock(self, index, action):
    # 可用股数
    available_share = self.share_list[index]
    # 卖出价
    sell_price = self.price_list[index]

    if available_share <= 0:
      return 0
    if sell_price <= 0:
      return 0

    # 计划卖出总金额 = 比例（abs（action））*帐号总金额
    pred_sell_amount = abs(action) * self.hmax
    # 卖出股数
    pred_sell_num_shares = 100 * (pred_sell_amount // (sell_price * 100))
    if self.mode == 'test':
      print("sell--stock_dim: ", index, " pred_sell_num_shares: ", pred_sell_num_shares, " available_share: ", available_share, " sell_price: ", sell_price)
    sell_num_shares = min(pred_sell_num_shares, available_share)
    # 金额
    commision = min(5, sell_price * sell_num_shares * self.sell_cost_pct)
    sell_amount = (sell_price * sell_num_shares * 0.999) - commision

    # 修改状态
    self.cash += sell_amount
    self.share_list[index] -= sell_num_shares
    self.cost += (sell_price * sell_num_shares * self.sell_cost_pct)
    self.trades += 1
    return sell_num_shares

  def _buy_stock(self, index, action):
    buy_price = self.price_list[index]
    if buy_price <= 0:
      return 0

    # 可以买的上限
    available_amount = self.cash // buy_price
    available_amount = (available_amount // 100) * 100
    if available_amount == 0:
      return 0

    # 计划买
    pred_buy_amount = abs(action) * self.hmax
    pred_buy_num_shares = 100 * (pred_buy_amount // (buy_price * 100))
    buy_num_shares = min(pred_buy_num_shares, available_amount)
    if self.mode == 'test':
      print("buy--stock_dim: ", index, " pred_buy_num_shares: ", pred_buy_num_shares, " available_amount: ", available_amount, " buy cash: ", self.cash, " buy price: ", buy_price)

    # 交易金额
    commision = min(5, buy_price * buy_num_shares * self.sell_cost_pct)
    buy_amount = buy_price * buy_num_shares + commision

    # 修改状态
    self.cash -= buy_amount
    self.share_list[index] += buy_num_shares
    self.cost += buy_price * buy_num_shares * self.buy_cost_pct
    self.trades += 1
    return buy_num_shares

  def step(self, actions):
    if self.mode == 'test':
      print("")
    # 判断是否合法
    actions = np.array([actions*0.2-1])
    self.fund_rate = 0

    # 是否是数据的最后一天
    self.terminal = self.day >= len(self.df.index.unique()) - 1
    if self.terminal:
      if self.make_plots:
        self._make_plot()

      # 存储最后一天的action
      self._update_actions(actions)
      self._save_action_memory_at_last()
      self._save_asset_memory_at_last()

      end_total_asset = self.cash + sum(np.array(self.price_list)* np.array(self.share_list))
      tot_reward = end_total_asset - self.initial_amount
      df_total_value = pd.DataFrame(self.asset_memory)
      df_total_value.columns = ["account_value"]
      df_total_value["date"] = self.date_memory
      df_total_value["daily_return"] = df_total_value["account_value"].pct_change(1)
      sharpe = self.get_sharpe()
      df_rewards = pd.DataFrame(self.rewards_memory)
      df_rewards.columns = ["account_rewards"]
      df_rewards["date"] = self.date_memory[:-1]
      self.fund_rate = tot_reward * (250/self.day) * 100/ self.initial_amount
      self.reward = 0
      # if self.episode % self.print_verbosity == 0:
      #   print(f"day: {self.day}, episode: {self.episode}")
      #   print(f"fund rate yearly: {self.fund_rate:0.2f}%")
      #   print(f"total_reward: {tot_reward:0.2f}")
      #   print(f"total_cost: {self.cost:0.2f}")
      #   print(f"total_trades: {self.trades}")
      #   if df_total_value["daily_return"].std() != 0:
      #       print(f"Sharpe: {sharpe:0.3f}")
      #   print("=================================")

    else:
      begin_total_asset = self.cash + sum(np.array(self.close_price_list)* np.array(self.share_list))
      yesterday_close = np.array(self.close_price_list)
      yesterday_share = np.array(self.share_list)
      yesterday_cash = self.cash
      # print(self.close_price_list, begin_total_asset)
      # 修改价格到下一天
      self.day += 1
      self.data = self.df.loc[self.day, :]
      self.state = self._update_state()

      # 使用第2天开盘价，actions执行交易
      self._update_actions(actions)
      end_total_asset = self.cash + sum(np.array(self.close_price_list)* np.array(self.share_list))
      today_close = np.array(self.close_price_list)
      today_share = np.array(self.share_list)
      today_cash = self.cash
      self.asset_memory.append(end_total_asset)
      self.date_memory.append(self._get_date())

      # 计算reward
      # self.reward = self.get_reward(end_total_asset, begin_total_asset, self.day)
      self.reward = self._get_reward(yesterday_close, yesterday_share, yesterday_cash, today_close, today_share, today_cash)
      self.rewards_memory.append(self.reward)
      # print(self.close_price_list, end_total_asset, self.reward)

    total_asset = self.cash + sum(np.array(self.price_list)* np.array(self.share_list))
    for i in range(self.stock_dim):
      self.state[i] = self.price_list[i] * self.share_list[i] / total_asset
    self.update_action_masks()
    return self.state, self.reward, self.terminal, {}

  def _get_reward(self, yesterday_close, yesterday_share, yesterday_cash, today_close, today_share, today_cash):
    if self.day == 1:
      if self.mode == 'test':
        print(yesterday_close, yesterday_share, yesterday_cash, today_close, today_share, today_cash, 0)
      return 0

    begin = yesterday_cash + sum(yesterday_share * yesterday_close)
    end = today_cash + sum(today_share * today_close)
    delta_share = np.negative(today_share - yesterday_share)
    delta_price = today_close - yesterday_close
    delta_assets = sum(-delta_share * delta_price)

    # assets = end - begin + delta_assets
    assets = end - begin
    reward = 100 * assets / self.initial_amount
    if self.mode == 'test':
      print(yesterday_close, yesterday_share, yesterday_cash, today_close, today_share, today_cash, begin, end, delta_assets, reward)
    return reward 

  def get_sharpe(self):
    df_total_value = pd.DataFrame(self.asset_memory)
    df_total_value.columns = ["account_value"]
    df_total_value["date"] = self.date_memory
    df_total_value["daily_return"] = df_total_value["account_value"].pct_change(1)

    sharpe = 0.0
    if df_total_value["daily_return"].std() != 0:
      sharpe = (
        (250 ** 0.5)
        * df_total_value["daily_return"].mean()
        / df_total_value["daily_return"].std()
      )
    return sharpe

  def _update_actions(self, actions):
    if self.mode == 'test':
      print("actions ", actions)
    argsort_actions = np.argsort(actions)
    if self.mode == 'test':
      print("argsort_actions ", argsort_actions)

    sell_index = argsort_actions[: np.where((actions < 0))[0].shape[0]]
    buy_index = argsort_actions[::-1][: np.where((actions > 0))[0].shape[0]]
    if self.mode == 'test':
      print("sell_index: ", sell_index)
      print("buy_index: ", buy_index)

    for index in sell_index:
      actions[index] = self._sell_stock(index, actions[index]) * (-1)

    for index in buy_index:
      actions[index] = self._buy_stock(index, actions[index])
    if self.mode == 'test':
      print("b_s_actions ", actions)
    self.actions_memory.append(actions)

  def get_sb_env(self):
    if self.ncpu == 1:
      e = DummyVecEnv([lambda: self])
      obs = e.reset()
      return e, obs
    else:
      e = SubprocVecEnv([lambda: self for i in range(self.ncpu)])
      obs = e.reset()
      return e, obs

  def _initiate_state(self):
    if len(self.df.tic.unique()) > 1:
      # for multiple stock
      state = (
          [0] * self.stock_dim
          + sum([self.data[tech].values.tolist() for tech in self.tech_indicator_list],[],)
      )
      self.cash = self.initial_amount
      self.price_list = self.data.open.values.tolist()
      self.close_price_list = self.data.close.values.tolist()
      self.share_list = [0] * self.stock_dim
    else:
      # for single stock
      state = (
          [0] * self.stock_dim
          + sum([[self.data[tech]] for tech in self.tech_indicator_list], [])
      )
      self.cash = self.initial_amount
      self.price_list = [self.data.open]
      self.close_price_list = [self.data.close]
      self.share_list = [0] * self.stock_dim
    return state

  def _update_state(self):
    if len(self.df.tic.unique()) > 1:
      # for multiple stock
      state = (
        list(self.state[0: self.stock_dim])
        + sum([self.data[tech].values.tolist() for tech in self.tech_indicator_list],[],)
      )
      self.price_list = self.data.open.values.tolist()
      self.close_price_list = self.data.close.values.tolist()

    else:
      # for single stock
      state = (
        list(self.state[0: self.stock_dim])
        + sum([[self.data[tech]] for tech in self.tech_indicator_list], [])
      )
      self.price_list = [self.data.open]
      self.close_price_list = [self.data.close]
    return state

  def _save_action_memory_at_last(self):
    if len(self.df.tic.unique()) > 1:
      date_list = self.date_memory
      df_date = pd.DataFrame(date_list)
      df_date.columns = ["date"]

      action_list = self.actions_memory
      df_actions = pd.DataFrame(action_list)
      df_actions.columns = self.data.tic.values
      df_actions.index = df_date.date
      self.df_actions = df_actions
    else:
      date_list = self.date_memory
      action_list = self.actions_memory
      df_actions = pd.DataFrame({"date": date_list, "actions": action_list})
      self.df_actions = df_actions

  def _save_asset_memory_at_last(self):
    date_list = self.date_memory
    asset_list = self.asset_memory
    self.df_account_value = pd.DataFrame(
        {"date": date_list, "account_value": asset_list}
    )
    
  def save_asset_memory(self):
    return self.df_account_value

  def save_action_memory(self):
    return self.df_actions

# 训练

## network

In [ ]:
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

import gym
import torch as th
from torch import nn

from stable_baselines3.common.policies import ActorCriticPolicy
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy



class CustomNetwork(nn.Module):
  def __init__(
    self,
    feature_dim: int,
    last_layer_dim_pi: int = 16,
    last_layer_dim_vf: int = 16,
  ):
    super(CustomNetwork, self).__init__()

    self.latent_dim_pi = last_layer_dim_pi
    self.latent_dim_vf = last_layer_dim_vf

    # Policy network
    self.policy_net = nn.Sequential(
        nn.Linear(feature_dim, last_layer_dim_pi), nn.ReLU()
    )
    # Value network
    self.value_net = nn.Sequential(
        nn.Linear(feature_dim, last_layer_dim_vf), nn.ReLU()
    )

  def forward(self, features: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
    return self.policy_net(features), self.value_net(features)

  def forward_actor(self, features: th.Tensor) -> th.Tensor:
    return self.policy_net(features)

  def forward_critic(self, features: th.Tensor) -> th.Tensor:
    return self.value_net(features)


class CustomActorCriticPolicy(MaskableActorCriticPolicy):
  def _build_mlp_extractor(self) -> None:
    self.mlp_extractor = CustomNetwork(self.features_dim)


## callback

In [ ]:
from sb3_contrib.common.maskable.callbacks import MaskableEvalCallback
from stable_baselines3.common.vec_env import sync_envs_normalization
from sb3_contrib.common.maskable.evaluation import evaluate_policy

class MyMaskableEvalCallback(MaskableEvalCallback):
  def _on_step(self) -> bool:
    if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
      # Sync training and eval env if there is VecNormalize
      sync_envs_normalization(self.training_env, self.eval_env)

      # Reset success rate buffer
      self._is_success_buffer = []

      e_trade_gym = MyTradeEnv(df=trade, **env_kwargs)
      e_trade_gym.seed(seed)


      test_env, test_obs = e_trade_gym.get_sb_env()
      test_env = VecNormalize(test_env, norm_obs=True, norm_reward=False, clip_obs=10.)
      test_env.training = False
      test_env.norm_reward = False

      episode_rewards, episode_lengths = evaluate_policy(self.model, test_env, n_eval_episodes=1)

      mean_reward, std_reward = np.mean(episode_rewards), np.std(episode_rewards)
      mean_ep_length, std_ep_length = np.mean(episode_lengths), np.std(episode_lengths)
      self.last_mean_reward = mean_reward

      if self.verbose > 0:
        print(f"Eval num_timesteps={self.num_timesteps}, " f"episode_reward={mean_reward:.2f} +/- {std_reward:.2f}")
      # Add to current Logger
      self.logger.record("eval/mean_reward", float(mean_reward))

      if len(self._is_success_buffer) > 0:
        success_rate = np.mean(self._is_success_buffer)
        if self.verbose > 0:
          print(f"Success rate: {100 * success_rate:.2f}%")
        self.logger.record("eval/success_rate", success_rate)

      # Dump log so the evaluation results are printed with the correct timestep
      self.logger.record("time/total timesteps", self.num_timesteps, exclude="tensorboard")
      self.logger.dump(self.num_timesteps)

      if mean_reward > self.best_mean_reward:
        if self.verbose > 0:
          print("New best mean reward!")
        if self.best_model_save_path is not None:
          self.model.save(os.path.join(self.best_model_save_path, "best_model"))
        self.best_mean_reward = mean_reward
        # Trigger callback if needed
        if self.callback is not None:
          return self._on_event()

    return True

## model

In [ ]:
from sb3_contrib import MaskablePPO
from stable_baselines3.common.type_aliases import MaybeCallback
from stable_baselines3.common.vec_env import VecEnv
from stable_baselines3.common.callbacks import BaseCallback, CallbackList, ConvertCallback

class MyMaskablePPO(MaskablePPO):
  def _init_callback(
    self,
    callback: MaybeCallback,
    eval_env: Optional[VecEnv] = None,
    eval_freq: int = 10000,
    n_eval_episodes: int = 5,
    log_path: Optional[str] = None,
    use_masking: bool = True,
  ) -> BaseCallback:

    if isinstance(callback, list):
      callback = CallbackList(callback)

    # Convert functional callback to object
    if not isinstance(callback, BaseCallback):
      callback = ConvertCallback(callback)

    callback.init_callback(self)
    return callback

## train

In [ ]:
train = data_split(processed_full, train_start_dt, train_end_dt)
trade = data_split(processed_full, valid_start_dt, valid_end_dt)
print(len(train))
print(len(trade))
stock_dimension = len(train.tic.unique())
state_space = stock_dimension + len(indicator_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [ ]:

env_kwargs = {
  "initial_amount": 100000, 
  "buy_cost_pct": 0.00025,
  "sell_cost_pct": 0.00025,
  "state_space": state_space, 
  "stock_dim": stock_dimension, 
  "tech_indicator_list": indicator_list, 
  "n_action_space": 11,
  "reward_scaling": 1,
  "ncpu": 1,
  "hmax": 100000, 
  # "mode": "test"
}

e_train_gym = MyTradeEnv(df=train, **env_kwargs)
e_train_gym.seed(seed)
env_train, _ = e_train_gym.get_sb_env()
env_train = VecNormalize(env_train, norm_obs=True, norm_reward=True, clip_obs=10.)


model_kwargs = {
  "n_steps": 2048,
  "ent_coef": 0.01,
  "learning_rate": 0.00025,
  "batch_size": 128,
}


#########
# trained_model.save("./model")

In [ ]:
# 迭代找最优
from finrl.agents.stablebaselines3.models import TensorboardCallback

# train
model_name = "ppo"

best_seed = 0
best_reward = -100
for seed in range(100,200):
  # test
  trained_model = MyMaskablePPO(policy=CustomActorCriticPolicy,
            env=env_train,
            tensorboard_log="tb_log",
            verbose=0,
            seed=seed,
            create_eval_env=True,
            **model_kwargs,
          )
  trained_model.learn(total_timesteps=10240 * 20,
            tb_log_name=model_name,
            callback=[
              TensorboardCallback(),
              MyMaskableEvalCallback(env_train,n_eval_episodes=1,eval_freq=10240*1
                          ,log_path="new_eval_log_path_{}".format(seed)
                          ,best_model_save_path="new_eval_log_path_{}".format(seed))
            ],
          )
  # evaluate
  e_trade_gym = MyTradeEnv(df=trade, **env_kwargs)
  e_trade_gym.seed(seed)
  test_env, test_obs = e_trade_gym.get_sb_env()
  test_env = VecNormalize(test_env, norm_obs=True, norm_reward=False, clip_obs=10.)
  test_env.training = False
  test_env.norm_reward = False

  model_path = "new_eval_log_path_{}/best_model.zip".format(seed)
  trained = MyMaskablePPO.load(model_path)
  mean_reward, std_reward = evaluate_policy(trained, test_env, n_eval_episodes=1)
  if mean_reward > best_reward:
    best_reward = mean_reward
    best_seed = seed
  print("best_seed ", best_seed, "best_reward ", best_reward)
  print("next_round")

# 回测

In [ ]:
# model_path = "./model"
seed = 0
# model_path = "new_eval_log_path_{}/best_model.zip".format(seed)
model_path = "best_model.zip"
trained = MyMaskablePPO.load(model_path)
e_trade_gym = MyTradeEnv(df=trade, **env_kwargs)
e_trade_gym.seed(seed)


test_env, test_obs = e_trade_gym.get_sb_env()
# test_env = VecNormalize.load("./env.pkl", test_env)
test_env = VecNormalize(test_env, norm_obs=True, norm_reward=False, clip_obs=10.)
test_env.training = False
test_env.norm_reward = False

account_memory = []
actions_memory = []
rewards_list = []
test_env.reset()
for i in range(len(e_trade_gym.df.index.unique())):
  action_masks = e_trade_gym.action_masks()
  action, _states = trained.predict(test_obs,deterministic=True,action_masks=action_masks)
  test_obs, rewards, dones, info = test_env.step(action)
  rewards_list.append(rewards[0])
  if dones[0]:
    actions_memory = test_env.env_method(method_name="save_action_memory")
    account_memory = test_env.env_method(method_name="save_asset_memory")
    print("hit end!")
    break

df_account_value, df_actions = account_memory[0], actions_memory[0]

In [ ]:
e_trade_gym = MyTradeEnv(df=trade, **env_kwargs)
e_trade_gym.seed(seed)


test_env, test_obs = e_trade_gym.get_sb_env()
test_env = VecNormalize(test_env, norm_obs=True, norm_reward=False, clip_obs=10.)
test_env.training = False
test_env.norm_reward = False

# mean_reward, std_reward = evaluate_policy(trained, test_env, n_eval_episodes=10)
mean_reward, std_reward = evaluate_policy(trained, test_env, n_eval_episodes=1)
mean_reward

In [ ]:
df_account_value

In [ ]:

display;
if len(cnstock) != 1:
  print(df_actions.sum())
  print(df_account_value.tail(1))
  display = df_actions.cumsum().drop_duplicates()
  display.index = pd.to_datetime(display.index)
else:
  df_actions['actions'] = df_actions['actions'].apply(lambda x: x[0])
  display = df_actions.copy();
  display['actions'] = display['actions'].cumsum()
  display = display.set_index(['date'],drop=True)
  df_actions = df_actions.set_index(['date'],drop=True)
display

## pyfolio

In [ ]:
%matplotlib inline
baseline_ticker = "000001.ss"
if len(cnstock) == 1:
  baseline_ticker = cnstock[0]

backtest_plot(df_account_value, 
      baseline_ticker = baseline_ticker, 
      baseline_start = valid_start_dt,
      baseline_end = valid_end_dt)

## matplot

In [ ]:
def plot(tradedata,actionsdata,ticker):    
  fig=plt.figure(figsize=(12, 6))
  ax=fig.add_subplot(211) 
  df_plot = pd.merge(left=tradedata ,right=actionsdata,on='date',how='inner')
  if len(cnstock) == 1:
    plot_df = df_plot.loc[df_plot['tic']==ticker].loc[:,['date','tic','open','actions']].reset_index()
  else:
    plot_df = df_plot.loc[df_plot['tic']==ticker].loc[:,['date','tic','open',ticker]].reset_index()

  print(plot_df)
  plot_df['datetime'] = pd.to_datetime(plot_df['date'], format="%Y-%m-%d")
  ax.plot(plot_df.datetime, plot_df['open'], label=ticker)
  locator = mdates.MonthLocator()
  ax.xaxis.set_major_locator(locator)
  fig.autofmt_xdate()
  # 主图 
  if len(cnstock) == 1:
    ax.plot(plot_df.loc[plot_df['actions']>0].datetime, plot_df['open'][plot_df['actions']>0], label='Buy', linewidth=0, marker='^', c='g')
    ax.plot(plot_df.loc[plot_df['actions']<0].datetime, plot_df['open'][plot_df['actions']<0], label='Sell', linewidth=0, marker='v', c='r')
  else:
    ax.plot(plot_df.loc[plot_df[ticker]>0].datetime, plot_df['open'][plot_df[ticker]>0], label='Buy', linewidth=0, marker='^', c='g')
    ax.plot(plot_df.loc[plot_df[ticker]<0].datetime, plot_df['open'][plot_df[ticker]<0], label='Sell', linewidth=0, marker='v', c='r')
  # 副图
  if len(cnstock) == 1:
    ax2 = plt.subplot(212, sharex=ax)
    ax2.plot(pd.to_datetime(display.index), display['actions'])
  else:
    ax2 = plt.subplot(212, sharex=ax)
    ax2.plot(display.index, display[ticker])
  plt.legend(loc='best')
  plt.grid(True)
  plt.title(ticker +'__'+plot_df['date'].min()+'___'+plot_df['date'].max())
  plt.show()
  if len(cnstock) == 1:
    print(plot_df.loc[plot_df['actions']!=0])
  else:
    print(plot_df.loc[plot_df[ticker]!=0])

for tic in cnstock:
  plot(trade,df_actions,tic)

In [ ]:
df_plot = pd.merge(left=trade ,right=df_actions,on='date',how='inner')
df_plot.loc[df_plot['tic']==cnstock[0]].loc[:,['date','tic','open','actions']].reset_index()